In [27]:
%load_ext autoreload

%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [28]:
from experiment import experiment

In [29]:
import os

In [30]:
import pandas as pd

In [31]:
results = os.listdir('results')

In [32]:
results = [result for result in results if result.endswith('.pkl') and "6.05.2023" in result and "rfqr_20runs" in result and "std" in result and "B=100" in result and "logFalse" in result and "randomConformalFalse" in result]


In [33]:
results

['bike_rfqr_20runs_1fraction_logFalse_randomConformalFalse_cqrbaggstd_twosidedconformalFalse_B=100_06.05.2023.pkl',
 'community_rfqr_20runs_1fraction_logFalse_randomConformalFalse_cqrbaggstd_twosidedconformalFalse_B=100_06.05.2023.pkl',
 'bio_rfqr_20runs_0.25fraction_logFalse_randomConformalFalse_cqrbaggstd_twosidedconformalFalse_B=100_06.05.2023.pkl',
 'concrete_rfqr_20runs_1fraction_logFalse_randomConformalFalse_cqrbaggstd_twosidedconformalFalse_B=100_06.05.2023.pkl',
 'homes_rfqr_20runs_0.5fraction_logFalse_randomConformalFalse_cqrbaggstd_twosidedconformalFalse_B=100_06.05.2023.pkl',
 'star_rfqr_20runs_1fraction_logFalse_randomConformalFalse_cqrbaggstd_twosidedconformalFalse_B=100_06.05.2023.pkl']

In [34]:
output = pd.DataFrame(columns=['UACQR-P','UACQR-S', 'CQR','CQR-r', 'UACQR-P SE', 'UACQR-S SE', 'CQR SE','CQR-r SE'])
for i,result in enumerate(results):
    title = results[i].split('_')[0] if results[i].split('_')[0] not in ["facebook","meps"] else '_'.join(results[i].split('_')[0:2])
    result = experiment(file_name='results/'+results[i])
    result.plot(calc_only=True)
    output.loc[title,['UACQR-P','UACQR-S', 'CQR','CQR-r']] = result.mean_results.min()
    output.loc[title,['UACQR-P SE','UACQR-S SE', 'CQR SE','CQR-r SE']] = result.sem_results.stack().loc[pd.MultiIndex.from_frame(result.mean_results.idxmin().rename('min_samples_leaf').reset_index().iloc[:,::-1])].to_numpy()
    
    

In [36]:
from functools import partial

In [37]:
def format_fig_and_se(row, method='CQR'):
    fig = row[method]
    se = row[method+' SE']
    if fig >1000:
        return f"{fig:.0f} ({se:.0f})"
    elif fig >1:
        return f"{fig:.3f} ({se:.3f})"
    else:
        return f"{fig:.3f} ({se:.3f})"

In [38]:
drop_columns = [c for c in output.columns if 'SE' in c]

In [39]:
output['CQR'] = output.apply(partial(format_fig_and_se, method='CQR'), axis=1)
output['UACQR-S'] = output.apply(partial(format_fig_and_se, method='UACQR-S'), axis=1)
output['UACQR-P'] = output.apply(partial(format_fig_and_se, method='UACQR-P'), axis=1)
output['CQR-r'] = output.apply(partial(format_fig_and_se, method='CQR-r'), axis=1)

In [40]:
output.drop(columns=drop_columns, inplace=True)

In [41]:
# Export to LaTeX
latex_table = output.sort_index().style.to_latex()


print('\_'.join(latex_table.split('_')))

\begin{tabular}{lllll}
 & UACQR-P & UACQR-S & CQR & CQR-r \\
bike & 1.155 (0.009) & 1.452 (0.008) & 1.481 (0.008) & 1.436 (0.008) \\
bio & 1.548 (0.009) & 1.650 (0.006) & 1.652 (0.006) & 1.649 (0.006) \\
community & 1.527 (0.022) & 1.752 (0.017) & 1.762 (0.015) & 1.759 (0.016) \\
concrete & 0.691 (0.010) & 0.734 (0.008) & 0.750 (0.007) & 0.727 (0.008) \\
homes & 0.634 (0.004) & 0.693 (0.005) & 0.754 (0.005) & 0.708 (0.005) \\
star & 0.177 (0.001) & 0.176 (0.001) & 0.175 (0.001) & 0.176 (0.001) \\
\end{tabular}

